## Align reads to genome using `HISAT2`

In [ ]:
REF_DIR="~/data1/ref/Caenorhabditis_elegans"
BASE_DIR="../data"
INDEX="$REF_DIR/wbcel235/genome"

hisat_exe="$HOME/bin/hisat2-2.1.0/hisat2"

for sample in $(ls $BASE_DIR | grep ^ig)
do
    sample_dir="$BASE_DIR/$sample"
    result_dir="../results/$sample"
    
    if [ ! -d "$result_dir" ]; then
        echo "Creating $result_dir ..."
        mkdir -p $result_dir
    fi
    
    echo "Unzipping reads for $sample ..."
    read1=$sample_dir/read1.fifo
    read2=$sample_dir/read2.fifo
    mkfifo $read1
    mkfifo $read2
    
    zcat $sample_dir/R1.fastq.gz > $read1 &
    zcat $sample_dir/R2.fastq.gz > $read2 &
    
    $hisat_exe -p 30 --dta --quiet \
        -x $INDEX \
        -1 $read1 -2 $read2 \
    | samtools view -bhS - > "$result_dir/$sample.bam"
    
    rm $sample_dir/*.fifo 
    
done

In [ ]:
for sample in $(ls $BASE_DIR | grep ^ig)
do
    result_dir="../results/$sample"
    samtools sort -@ 30 -o "$result_dir/$sample.sorted.bam" "$result_dir/$sample.bam"
    samtools index -@ 30 "$result_dir/$sample.sorted.bam"
done